In [ ]:
#input date and time
from datetime import datetime
from obspy import UTCDateTime
import requests
import os
from bs4 import BeautifulSoup
# Input values
year = int(input('year: '))
month = int(input('month: '))
day = int(input('day: '))
hour = int(input('hour: '))
minute = int(input('minute: '))
second = int(input('second: '))


# Convert the datetime to UTCDateTime (for both UTC and DOY)
dt = datetime(year, month, day, hour, minute, second)

# Convert to UTC and format as ISO 8601 with "Z"
utc_time = dt.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + "Z"
print("UTC Time:", utc_time)

# Calculate the day of the year (DOY)
doy = dt.timetuple().tm_yday
print("Day of Year (DOY):", doy)
os.makedirs('//home//ajohns//Downloads//'+str(dt), exist_ok=True)
path = '//home//ajohns//Downloads//'+str(dt)
print(utc_time)

In [ ]:
def download_file(file_url, type):
    response = requests.get(file_url)
    file_name = file_url.split('/')[-1]
    os.makedirs(f'{path}//{type}_data', exist_ok=True)
    file_path = os.path.join(f'{path}//{type}_data', file_name)
    
    with open(file_path, 'wb') as file:
        file.write(response.content)
        print(f'Downloaded: {file_name} to Mseed_data/')

In [ ]:
#search for seismic wave file

# Base URL for the continuous waveform data
base_url = 'https://pds-geosciences.wustl.edu/insight/urn-nasa-pds-insight_seis/data/xb/continuous_waveform/elyhk/' + str(year) +'/' + str(doy) + '/'
# Create directories for each file type

def extract_data(url):
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a', href=True)

    for link in links:
        href = link['href']
        cond = 0
        #print('the link is : ',href)
        full_url = 'https://pds-geosciences.wustl.edu'+href
        #print(f"Checking URL: {full_url}")  # Debug print
        if link.text == '[To Parent Directory]' or href == 'https://www.nasa.gov/':
            cond = 'error'
        else:
            cond = 'success'
        # Check if the link is a directory (if so, recurse into it)
        
        if href.endswith('.mseed'):
            file_response = requests.get(full_url)
            if file_response.status_code == 200:
                print(href.split('.')[3][0:2])
                download_file(full_url, 'mseed')
            else:
                print(f"Failed to access {full_url}: Status code {file_response.status_code}")
        elif cond == 'success':
            extract_data(full_url)

# Start the extraction process
extract_data(base_url)

In [ ]:
#search for twins file

y = open('//home//ajohns//Downloads//newer_output.csv', 'r')
z = y.readlines()
from datetime import datetime, timedelta
current_date = datetime(year, month, day)
next_day = current_date + timedelta(days=1)
current_date = str(current_date).split()[0]
next_day = str(next_day).split()[0]
print(current_date, next_day)
file_list = []
for c in z:
    c = c.split(',')
    try:
        if c[1] == current_date:
            file_list.append([c[2], c[3]])
        elif c[1] == next_day:
            file_list.append([c[2], c[3]])
            break
    except:
        break
for x in file_list:
    print(x)
    download_file('https://atmos.nmsu.edu/PDS/data/PDS4/InSight/twins_bundle/data_calibrated/' + x[1][:-1] + '/' + x[0]+ '.csv', 'csv')

In [ ]:
#plot the msee ddata for the specific day

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from obspy import read

mseed_folder = os.path.join(path, 'mseed_data')
csv_folder = os.path.join(path, 'mseed_data')

fig, ax = plt.subplots(figsize=(10, 6))
def mplot(mseed):
    st = read(mseed)
    trace = st[0]
    times = trace.times("matplotlib")

    ax.plot(times, trace.data, label='Seismic Trace')
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
    plt.xticks(rotation=45)

for x in os.listdir(mseed_folder):
    mplot(os.path.join(mseed_folder, x))

plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Seismic Trace with Time')

specific_time = dt

# Add a vertical line at the specific time
ax.axvline(x=specific_time, color='red', linestyle='--', linewidth=2, label='Event Time')
plt.tight_layout()
plt.show()